# Sentimendianalüüs


## Sisukord

* [Ülesanne 9 ja Boonusülesanne](#9)
* [Andmete lugemine](#lugemine)
* [Sõnade relevantsuse hindamine](#relevants)
* [Logistiline regressioon dokumentide klassifitseerimiseks](#klf)

Põhineb S.Raschka *Python Machine Learnig* raamatu 
[peatükil 8](https://github.com/rasbt/python-machine-learning-book/blob/master/code/ch08) ([*MIT litsents*](https://github.com/rasbt/python-machine-learning-book/blob/master/LICENSE.txt)).

<a id='9'></a>


## Ülesanne 9.BOONUS

Koostada oma originaalandmetest (vt ka Moodle sektsiooni *Iseseisev töö*) .csv fail. Kirjeldada andmestikku lühidalt ja dokumenteerida andmestiku atribuudid eraldi failis kujul **atribuudi_nimi, andmetüüp (tõeväärtus, kategooria, täisarv, reaalarv, tekst, kuupäev,...), selgitus**. Kui on olemas klass või arvväärtus, mida soovime ennustada, siis dokumenteerida ka see.

Viige oma originaalandmestik sellisele kujule, mida ka teised tudengid saaksid kasutada ja laadige see Moodlesse üles. Vaadake, et teil oleksid seaduslikud  ja tööalased õigused neid andmeid jagada. Anonümiseerige isikuandmed (kui on) st asendage nimed ja isikukoodid meelevaldsete koodidega.

See läheb kirja ka kui ülesande 9. esitamine, mida lisaks pole vaja teha. 

Kuigi iseseisvas töös on lubatud kasutada avalikke andmeid ka [Kagglest](https://www.kaggle.com/datasets) ja muudest allikatest, siis selle ülesandena lähevad kirja ainult andmed, mis pole avalikult saadaval või mis on saadud avalikest andmetest töötlemise abil. Näiteks Kagglest võetud .csv faili muutmata kujul esitamine ei ole selle boonusülesande mõte.


## Ülesanne 9

Kui originaalandmeid ei leia:

Laadida Project Gutenberg lehelt alla Goethe Fausti http://www.gutenberg.org/ebooks/14591 ja Iliase http://www.gutenberg.org/ebooks/6130 tekstifailid (*plain text, utf-8*). Jagada need failid salmideks, kus salme eraldavad kaks reavahetust ja salmipikkus on üle saja ja alla tuhande tähemärgi, näiteks:

`text = file.read()
salmid = [t for t in text.split("\n\n") if 100 < len(t) < 1000]`

Koostage .csv fail, kus read on salmid ja esimeseks veeruks on salmi tekst ning teiseks veeruks on salmi allikas (Ilias või Faust).

In [7]:
out_csv = []
with open('./faust.txt') as f:
    text = f.read().replace(',', '')
    out_csv += [(v.replace('\n', ''), 'faust') for v in text.split('\n\n') if 100 < len(v) < 1000]
    
with open('./ilias.txt') as f:
    text = f.read().replace(',', '')
    out_csv += [(v.replace('\n', ''), 'ilias') for v in text.split('\n\n') if 100 < len(v) < 1000]

with open('./verses.csv', 'w+') as f:
    f.write('\n'.join([','.join(x) for x in out_csv]))
    

<a id='lugemine'></a>
## Andmete lugemine

Andmete töötluseks ja movie_data.csv faili koostamiseks on eraldi ipynb märkmik.

In [44]:
import numpy as np
import pandas as pd
import math
from sklearn.feature_extraction.text import CountVectorizer


In [45]:
# Eeldus: movie_data.csv on tekitatud

df = pd.read_csv('./movie_data.csv')
df.head(3)


,review,sentiment
0,I thought this was one of those really great f...,1
1,PLEASE people! DO NOT bother with this poorly ...,0
2,"I must tell you right up front, I am certainly...",0


<a id='relevants'></a>
## Sõnade relevantsuse hindamine

Terminisagedus $tf(t, d)$ näitab kui mitu korda termin $t$ esineb dokumendis $d$. Klassi [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) transformeerimismeetod `fit_transform(X)` teisendab tekstimassiivi $t \times d$ terminisagedusmassiiviks.

In [46]:
count = CountVectorizer()
docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining and weather is sweet'])
bag = count.fit_transform(docs)

print(count.vocabulary_)
print(bag.toarray())

{'the': 5, 'sun': 3, 'is': 1, 'shining': 2, 'weather': 6, 'sweet': 4, 'and': 0}
[[0 1 1 1 0 1 0]
 [0 1 0 0 1 1 1]
 [1 2 1 1 1 1 1]]


Sõnad, mis esinevad peaaegu kõigis dokumentides ei ole tavaliselt   dokumentide eristamise mõttes kuigi informatiivsed. Mõõt idf (*inverse document frequency*)  on logaritm dokumentide arvu $n_d$ suhtest sõna $t$ sisaldavate dokumentide arvu $f_d(t)$ ja on seega seda suurem, mida vähemates dokumentides sõna esineb. Vahel modiitseeritakse seda funktsiooni liites jagajale või jagatavale arvu 1.

$$ idf (t) = log \frac{n_d}{f_d(t)} $$

Mõõt **tf-idf** (*term frequency - inverse document frequency*) korrigeerib terminisagedust $tf$ mõõduga $idf$

$$ tfidf (t, d) = tf(i, d) \times idf(t) $$

In [47]:
np.set_printoptions(precision=2)

In [48]:
X = bag.toarray()

def idf(X):
    """ 
    Leiame idf  mõõtude vektori kõigile sõnadele kui meie 
    sagedusmaatriks (bag.toarray()) on X.
    """
    n_d = len(X)
    f_d = np.sum(X != 0, axis=0)
    #print(f_d)
    return np.log(n_d / f_d)

idf(X)


array([1.1 , 0.  , 0.41, 0.41, 0.41, 0.  , 0.41])

In [49]:
def tfidf(X):
    """ 
    Leiame tfidf  sageduste vektori kõigile sõnadele kui meie 
    sagedusmaatriks (bag.toarray()) on X.
    """
    return X * idf(X)

tfidf(X)

array([[0.  , 0.  , 0.41, 0.41, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.41, 0.  , 0.41],
       [1.1 , 0.  , 0.41, 0.41, 0.41, 0.  , 0.41]])

Moodul sklearn pakub selleks klassi [TfidfTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html). Tulemused erinevad seoses funktsioonide veidi  erinevate definitsioonidega 

$$ idf (t) = log \frac{1+n_d}{1+f_d(t)} $$


$$ tfidf (t, d) = tf(i, d) \times (idf(t) + 1) $$

ja dokumendi sagedusvektori normaliseerimisega ühikpikkuseks (L2-normaliseerimine).

In [50]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_t = TfidfTransformer()
TFIDF_X = tfidf_t.fit_transform(count.fit_transform(docs)).toarray()
TFIDF_X

array([[0.  , 0.43, 0.56, 0.56, 0.  , 0.43, 0.  ],
       [0.  , 0.43, 0.  , 0.  , 0.56, 0.43, 0.56],
       [0.44, 0.53, 0.34, 0.34, 0.34, 0.26, 0.34]])

In [51]:
np.sum(TFIDF_X**2, axis=1)

array([1., 1., 1.])

<a id='klf'></a>
## Logistiline regressioon dokumentide klassifitseerimiseks





Klass [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) lihtsustab tf-idf mõõdu leidmist olles samaväärne  [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) ja [TfidfTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html) üksteise järel rakendamisega. Alustuseks leiame hüperparameetrite otsinguga [GrisSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.htm) parimad parameetrid. Kasutame selleks ainult 5000 objekti, muidu oleks see samm väga ajamahukas.

In [52]:
len(df)

50000

In [53]:
X_train = df.loc[:5000, 'review'].values
y_train = df.loc[:5000, 'sentiment'].values
X_test = df.loc[5000:10000, 'review'].values
y_test = df.loc[5000:10000, 'sentiment'].values

In [54]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [55]:
print(ord("."))
"k,k,l,o,ooo.".translate({44: None, 46: "@"})

46


'kkloooo@'

In [56]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [57]:
#{ord(c):None for c in string.punctuation}

In [58]:
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

def tokenizer(text):return text.split()
def nopunctuation_tokenizer(text):
    np_text = text.translate({ord(c):None for c in string.punctuation})
    return np_text.split()

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__tokenizer': [tokenizer, nopunctuation_tokenizer],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__tokenizer': [tokenizer, nopunctuation_tokenizer],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, solver="liblinear"))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=1)

In [59]:
# Väga ajamahukas samm
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  2.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=False,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

In [60]:
print("Parimad parameetrid: ", gs_lr_tfidf.best_params_)
print("\nTäpsus: ", gs_lr_tfidf.best_score_)


Parimad parameetrid:  {'clf__C': 100.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__tokenizer': <function nopunctuation_tokenizer at 0x000001E915D682F0>}

Täpsus:  0.8582253746253746


Seejärel kasutame neid parameetreid ennustava mudeli treenimiseks andmestiku esimese 25 000 objekti peal ja leiame kõige suurema koefitsendiga (kõige kaalukamad) sõnad otsuse tegemisel.

In [61]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

lr_tfidf.set_params(**gs_lr_tfidf.best_params_)
lr_tfidf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                                 tokenizer=<function nopunctuation_tokenizer at 0x000001E915D682F0>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=100.0, class_weight=None, dual=False,
                                   

Teeme 10-kordse ristkontrolli.

In [62]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=lr_tfidf,
                         X=X_train,
                         y=y_train,
                         cv=10,
                         n_jobs=1)
print('CV täpsused: ', scores)
print('CV keskmine täpsus: %.3f' % np.mean(scores), "+/- %.3f" % np.std(scores))

CV täpsused:  [0.89 0.89 0.88 0.88 0.9  0.88 0.9  0.89 0.9  0.9 ]
CV keskmine täpsus: 0.892 +/- 0.007


Ja lõpuks kontrollime erinevate meetrikate ja testandmetega.

In [63]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix

y_pred = lr_tfidf.predict(X_test)
print("Täpsus testandmetel:", accuracy_score(y_test, y_pred))
print("F1-skoor testandmetel:", f1_score(y_test, y_pred))
print("Eksimismaatriks:")
print(confusion_matrix(y_test, y_pred))

Täpsus testandmetel: 0.89476
F1-skoor testandmetel: 0.8951082406410716
Eksimismaatriks:
[[11143  1405]
 [ 1226 11226]]


In [64]:
coef = lr_tfidf.named_steps['clf'].coef_[0]
print("Koefitsendid:", coef)

Koefitsendid: [-0.04 -4.86  1.09 ...  0.06 -0.01 -2.4 ]


In [65]:
word_dict = lr_tfidf.named_steps['vect'].vocabulary_
print("Sõnastik:", len(word_dict))

Sõnastik: 145431


In [66]:
term_coef = [(coef[word_dict[w]], w) for w in word_dict]
term_coef = sorted(term_coef, key=lambda pair: abs(pair[0]), reverse=True)
print("Need sõnad mõjutavad arvustuse sentimenti enim (koefitsent, sõna):\n")
term_coef[:20]

Need sõnad mõjutavad arvustuse sentimenti enim (koefitsent, sõna):



[(-30.163405140363256, 'worst'),
 (-23.484700985222013, 'awful'),
 (-21.420029416165136, 'waste'),
 (21.331065960953826, '710'),
 (18.210690229167227, 'excellent'),
 (-16.629842670432787, 'bad'),
 (-16.576548522894914, 'fails'),
 (-16.42485928512674, 'boring'),
 (16.272315235901335, 'great'),
 (-15.785860680556352, 'nothing'),
 (15.763725870990394, 'wonderful'),
 (-15.704029862739269, 'disappointing'),
 (-14.80804035948606, 'poorly'),
 (-14.598099772128512, 'poor'),
 (-14.326725256635525, 'worse'),
 (-14.226544332275056, 'terrible'),
 (-14.197902733280756, 'dull'),
 (13.881969626133381, '810'),
 (-13.855992129665413, 'disappointment'),
 (-13.708671589825316, 'forgettable')]